<center><h1>Capstone_project</h1></center>

## Where should I open a new restaurant?

## Part 1: Get neighborhoods data and prepare dataframe

### 1. Read neighborhoods data from wikipage

In [2]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text

In [5]:
soup = BeautifulSoup(data,"html5lib")

In [6]:
#find all html tables in the web page
tables = soup.find_all('table') # in html table is represented by the tag <table>

In [7]:
# we can see how many tables were found by checking the length of the tables list
len(tables)

3

### 2.Transfer xml neighborhoods table data to dataframe

In [8]:
table_contents=[]

for row in tables[0].findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

print(table_contents)

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

In [9]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
df.shape

(103, 3)

### 3. Get the csv file and transfer to a dataframe

In [11]:
url1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv"
df1 = pd.read_csv(url1)
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 4. Merge 2 dataframe by column PostalCode

In [12]:
neighborhood_data = pd.merge(df, df1, left_on = 'PostalCode', right_on = 'Postal Code')
neighborhood_data.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


### 5. Show the data overview

In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhood_data['Borough'].unique()),
        neighborhood_data.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


### 6. Create map overview of Toronto neighborhoods

In [14]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
#Filter only toronto data
toronto_data = neighborhood_data[neighborhood_data['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306


In [16]:
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

## Part 2: Get venues data from Foursquare

### 1. Get venues data from foursquare, and transfer the data info dataFrame

In [17]:
CLIENT_ID = 'KJJMV3FXF0MA2U3FMBJN1DBG4ZCMUQQ3PVBR15LAAP5FPOLL' # your Foursquare ID
CLIENT_SECRET = 'WTIS45L3G3SC0J1EUD2V5YEFWHG0S0MPCVN0GW2KW3BRYJY4' # your Foursquare Secret
VERSION = '20210515' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KJJMV3FXF0MA2U3FMBJN1DBG4ZCMUQQ3PVBR15LAAP5FPOLL
CLIENT_SECRET:WTIS45L3G3SC0J1EUD2V5YEFWHG0S0MPCVN0GW2KW3BRYJY4


In [18]:
toronto_data.shape

(39, 6)

**Create loop for getting different data for each neighborhoods**

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1572, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


Let's check how many venues were returned for each neighborhood

In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33
Davisville North,8,8,8,8,8,8


In [23]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 230 uniques categories.


In [24]:
toronto_venues.groupby('Venue Category').count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Adult Boutique,1,1,1,1,1,1
Airport,1,1,1,1,1,1
Airport Food Court,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2
...,...,...,...,...,...,...
Video Game Store,1,1,1,1,1,1
Vietnamese Restaurant,8,8,8,8,8,8
Wine Bar,9,9,9,9,9,9


In [28]:
test = toronto_venues
test.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [31]:
test[test['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
1,"Regent Park, Harbourfront",43.654260,-79.360636,El Catrin,43.650601,-79.358920,Mexican Restaurant
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
3,"Regent Park, Harbourfront",43.654260,-79.360636,Izumi,43.649970,-79.360153,Asian Restaurant
4,"Regent Park, Harbourfront",43.654260,-79.360636,Flame Shack,43.656844,-79.358917,Restaurant
...,...,...,...,...,...,...,...
364,Church and Wellesley,43.665860,-79.383160,A&W,43.666415,-79.378235,Fast Food Restaurant
365,Church and Wellesley,43.665860,-79.383160,Asahi Sushi,43.669874,-79.382943,Sushi Restaurant
366,Church and Wellesley,43.665860,-79.383160,McDonald's,43.668854,-79.385962,Fast Food Restaurant
367,Enclave of M4L,43.662744,-79.321558,Chick-n-Joy,43.665181,-79.321403,Fast Food Restaurant


### 2. Analyze Each Neighborhood and prepare the clustering

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
toronto_onehot.shape

(1572, 230)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each categor

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.016393,0.000000,0.000000,0.016393,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.025316,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,...,0.012658,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,...,0.000000,0.00000,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
toronto_grouped.shape

(38, 230)

Below is a function to sort the venues in descending order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Then to create the new dataframe and display the top 10 venues for each neighborhood.

In [31]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Cheese Shop,Pharmacy,Restaurant,Grocery Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Pet Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Grocery Store,Bakery
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Boutique,Rental Car Location,Plane,Coffee Shop,Sculpture Garden
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Yoga Studio,Discount Store
4,Christie,Grocery Store,Café,Park,Baby Store,Restaurant,Candy Store,Coffee Shop,Athletics & Sports,Italian Restaurant,Nightclub


In [32]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.016393,0.000000,0.000000,0.016393,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.025316,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,...,0.012658,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,...,0.000000,0.00000,0.030303,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


## Part 3. Cluster the neighborhoods

### 1. Do the cluster and make a overview map

In [33]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Then to create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [34]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,1.0,Coffee Shop,Park,Bakery,Restaurant,Theater,Café,Breakfast Spot,Pub,Mexican Restaurant,Farmers Market
1,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,1.0,Coffee Shop,Clothing Store,Italian Restaurant,Hotel,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Movie Theater
2,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,1.0,Café,Coffee Shop,Gastropub,Cosmetics Shop,Cocktail Bar,Cheese Shop,Department Store,Hotel,Farmers Market,Restaurant
3,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,1.0,Trail,Health Food Store,Pub,Wine Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
4,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,1.0,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Cheese Shop,Pharmacy,Restaurant,Grocery Store


In [55]:
toronto_merged.shape

(39, 17)

In [35]:
toronto_merged.dtypes

PostalCode                 object
Borough                    object
Neighborhood               object
Postal Code                object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [37]:
toronto_merged = toronto_merged.fillna(0)

In [38]:
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)

In [39]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 2. Examine Clusters

Cluster 1

In [94]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,-79.411307,0,Park,Trail,Jewelry Store,Sushi Restaurant,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
33,Downtown Toronto,-79.377529,0,Park,Trail,Playground,Deli / Bodega,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 2

In [95]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.360636,1,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Theater,Pub,Restaurant,Spa,Beer Store
1,Downtown Toronto,-79.378937,1,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Italian Restaurant,Japanese Restaurant,Bubble Tea Shop,Cosmetics Shop,Hotel,Electronics Store
2,Downtown Toronto,-79.375418,1,Café,Coffee Shop,Cocktail Bar,Cosmetics Shop,Gastropub,Restaurant,Cheese Shop,Farmers Market,Hotel,Department Store
3,East Toronto,-79.293031,1,Coffee Shop,Trail,Health Food Store,Pub,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,Downtown Toronto,-79.373306,1,Coffee Shop,Cocktail Bar,Bakery,Pharmacy,Farmers Market,Beer Bar,Seafood Restaurant,Restaurant,Cheese Shop,Juice Bar
5,Downtown Toronto,-79.387383,1,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Middle Eastern Restaurant,Restaurant,Bubble Tea Shop,Burger Joint,Japanese Restaurant
6,Downtown Toronto,-79.422564,1,Grocery Store,Café,Park,Nightclub,Coffee Shop,Restaurant,Candy Store,Baby Store,Italian Restaurant,Wine Bar
7,Downtown Toronto,-79.384568,1,Coffee Shop,Café,Restaurant,Thai Restaurant,Gym,Clothing Store,Deli / Bodega,Sushi Restaurant,Pizza Place,Salad Place
8,West Toronto,-79.442259,1,Bakery,Pharmacy,Café,Supermarket,Grocery Store,Music Venue,Park,Liquor Store,Brewery,Middle Eastern Restaurant
10,Downtown Toronto,-79.381752,1,Coffee Shop,Aquarium,Café,Hotel,Sporting Goods Shop,Italian Restaurant,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery


Cluster 3

In [96]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East York/East Toronto,-79.338106,2,Park,Convenience Store,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


Cluster 4

In [97]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,-79.416936,3,Home Service,Garden,Wine Bar,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Cluster 5

In [98]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,Central Toronto,-79.38316,4,Park,Restaurant,Lawyer,Gift Shop,German Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


## Part 4: Find a neighborhood in each cluster that suits the new restaurant best

### 1. Merge the dataframes so we can see total overview of the labels and restaurants

In [56]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636,1.0,Coffee Shop,Park,Bakery,Restaurant,Theater,Café,Breakfast Spot,Pub,Mexican Restaurant,Farmers Market
1,M5B,Downtown Toronto,"Garden District, Ryerson",M5B,43.657162,-79.378937,1.0,Coffee Shop,Clothing Store,Italian Restaurant,Hotel,Bubble Tea Shop,Cosmetics Shop,Middle Eastern Restaurant,Café,Japanese Restaurant,Movie Theater
2,M5C,Downtown Toronto,St. James Town,M5C,43.651494,-79.375418,1.0,Café,Coffee Shop,Gastropub,Cosmetics Shop,Cocktail Bar,Cheese Shop,Department Store,Hotel,Farmers Market,Restaurant
3,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,1.0,Trail,Health Food Store,Pub,Wine Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
4,M5E,Downtown Toronto,Berczy Park,M5E,43.644771,-79.373306,1.0,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Cheese Shop,Pharmacy,Restaurant,Grocery Store


In [63]:
toronto_merged.shape

(39, 17)

Drop redundant columns in dataframe toronto_merged

In [65]:
toronto_merged.drop(["PostalCode","Borough","Postal Code","Latitude","Longitude","1st Most Common Venue","2nd Most Common Venue","3rd Most Common Venue","4th Most Common Venue","5th Most Common Venue","6th Most Common Venue","7th Most Common Venue","8th Most Common Venue","9th Most Common Venue","10th Most Common Venue"], axis = 1, inplace=True)
toronto_merged.head()

,Neighborhood,Cluster Labels
0,"Regent Park, Harbourfront",1.0
1,"Garden District, Ryerson",1.0
2,St. James Town,1.0
3,The Beaches,1.0
4,Berczy Park,1.0


In [68]:
toronto_merged.groupby("Cluster Labels").count()

,Neighborhood
Cluster Labels,
0.0,2
1.0,33
2.0,1
3.0,1
4.0,1


**As we can see above, actually we do not have enough data in cluster label 2,3,4, so we will only check cluster label 0 and cluster label 1 in this project**

In [57]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [58]:
toronto_venues.shape

(1572, 7)

Add label column to toronto_venues, so we have an overview

In [69]:
neighborhood_final = pd.merge(toronto_venues, toronto_merged, left_on = 'Neighborhood', right_on = 'Neighborhood')
neighborhood_final.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,1.0
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,1.0
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,1.0
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,1.0
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,1.0


Drop redundant columns in neighborhood_final

In [70]:
neighborhood_final.shape

(1572, 8)

In [71]:
neighborhood_final.groupby("Cluster Labels").count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Cluster Labels,,,,,,,
0.0,8,8,8,8,8,8,8
1.0,1552,1552,1552,1552,1552,1552,1552
2.0,4,4,4,4,4,4,4
3.0,4,4,4,4,4,4,4
4.0,4,4,4,4,4,4,4


### 2. Drop cluster label = 2, 3, 4, as they do not have enough data to further analyze

In [79]:
neighborhood_final[neighborhood_final["Cluster Labels"].isin(["2.0","3.0","4.0"])]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels
947,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park,3.0
948,Lawrence Park,43.728020,-79.388790,HYC Design Inc.,43.726793,-79.391681,Business Service,3.0
949,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School,3.0
950,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line,3.0
951,Roselawn,43.711695,-79.416936,BAGEL TIME MONTREAL STYLE,43.709067,-79.415858,Fast Food Restaurant,2.0
952,Roselawn,43.711695,-79.416936,Ceiling Champions,43.713891,-79.420702,Home Service,2.0
953,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden,2.0
954,Roselawn,43.711695,-79.416936,Havergal College,43.712108,-79.411680,Music Venue,2.0
963,Forest Hill North & West,43.696948,-79.411307,Kay Gardner Beltline Trail,43.698446,-79.406873,Trail,4.0
964,Forest Hill North & West,43.696948,-79.411307,Nikko Sushi Japenese Restaurant,43.700443,-79.407957,Sushi Restaurant,4.0


In [80]:
index_names = neighborhood_final[neighborhood_final["Cluster Labels"].isin(["2.0","3.0","4.0"])].index

In [81]:
neighborhood_final.drop(index_names, inplace = True)

In [82]:
neighborhood_final.shape

(1560, 8)

Add column is_restaurant to prepare analysis afterwards

In [159]:
neighborhood_final['is_restaurant'] = np.where(neighborhood_final["Venue Category"].str.contains("Restaurant"), 1, 0)
neighborhood_final

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,is_restaurant
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,1.0,0
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,1.0,0
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,1.0,0
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,1.0,0
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,1.0,1
...,...,...,...,...,...,...,...,...,...
1567,Enclave of M4L,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa,1.0,0
1568,Enclave of M4L,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station,1.0,0
1569,Enclave of M4L,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop,1.0,0
1570,Enclave of M4L,43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center,1.0,0


In [161]:
neighborhood_final.shape

(1560, 9)

### 3. Make analysis on cluster label = 0 and 1, and see which neighborhood is the best choice to open a new restaurant

**First let's see label 0**

Check whole amounts of venues in cluster label = 0

In [162]:
#Assign label 0 data to a new dataframe
neighborhood_final_label0 = neighborhood_final[neighborhood_final["Cluster Labels"].isin(["0.0"])]
neighborhood_final_label0

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,is_restaurant
480,The Danforth East,43.685347,-79.338106,The Path,43.683923,-79.335007,Park,0.0,0
481,The Danforth East,43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store,0.0,0
482,The Danforth East,43.685347,-79.338106,Pizza Pizza,43.683065,-79.333846,Pizza Place,0.0,0
483,The Danforth East,43.685347,-79.338106,Donlands Subway Station,43.680960,-79.337759,Metro Station,0.0,0
1233,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground,0.0,0
1234,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park,0.0,0
1235,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park,0.0,0
1236,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail,0.0,0


In [163]:
toronto_counts = neighborhood_final_label0.groupby("Neighborhood").Neighborhood.agg("count").to_frame('total_venues_count_per_neighboorhood').reset_index()
toronto_counts

,Neighborhood,total_venues_count_per_neighboorhood
0,Rosedale,4
1,The Danforth East,4


Check how many restaurants are in cluser label 0

In [173]:
toronto_counts1 = neighborhood_final_label0.groupby("Neighborhood").sum("is_restaurant")
toronto_counts1.drop(["Neighborhood Latitude","Neighborhood Longitude","Venue Latitude","Venue Longitude","Cluster Labels"], axis=1, inplace=True)

In [174]:
toronto_counts1

,is_restaurant
Neighborhood,
Rosedale,0
The Danforth East,0


Merge the 2 count table and see final result

In [178]:
toronto_label0 = pd.merge(toronto_counts, toronto_counts1, left_on = 'Neighborhood', right_on = 'Neighborhood')
toronto_label0

,Neighborhood,total_venues_count_per_neighboorhood,is_restaurant
0,Rosedale,4,0
1,The Danforth East,4,0


Calculate the restaurants of venues percentage and see the order

In [179]:
toronto_label0["percentage"] = toronto_label0["is_restaurant"]/toronto_label0["total_venues_count_per_neighboorhood"]

**Show the final result of the percentage calculating and conclude the result**

In [182]:
toronto_label0.sort_values(by=["percentage"], ascending=True)

,Neighborhood,total_venues_count_per_neighboorhood,is_restaurant,percentage
0,Rosedale,4,0,0.0
1,The Danforth East,4,0,0.0


**Then let's see label 1**

In [183]:
#Assign label 1 data to a new dataframe
neighborhood_final_label1 = neighborhood_final[neighborhood_final["Cluster Labels"].isin(["1.0"])]
neighborhood_final_label1

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,is_restaurant
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,1.0,0
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,1.0,0
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center,1.0,0
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa,1.0,0
4,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,1.0,1
...,...,...,...,...,...,...,...,...,...
1567,Enclave of M4L,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa,1.0,0
1568,Enclave of M4L,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station,1.0,0
1569,Enclave of M4L,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop,1.0,0
1570,Enclave of M4L,43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center,1.0,0


Check whole amounts of venues in cluster label = 1

In [184]:
toronto_counts_1 = neighborhood_final_label1.groupby("Neighborhood").Neighborhood.agg("count").to_frame('total_venues_count_per_neighboorhood').reset_index()
toronto_counts_1

,Neighborhood,total_venues_count_per_neighboorhood
0,Berczy Park,58
1,"Brockton, Parkdale Village, Exhibition Place",22
2,"CN Tower, King and Spadina, Railway Lands, Har...",17
3,Central Bay Street,61
4,Christie,16
5,Church and Wellesley,79
6,"Commerce Court, Victoria Hotel",100
7,Davisville,33
8,Davisville North,8
9,"Dufferin, Dovercourt Village",15


Check how many restaurants are in cluser label 1

In [186]:
toronto_counts1_1 = neighborhood_final_label1.groupby("Neighborhood").sum("is_restaurant")
toronto_counts1_1.drop(["Neighborhood Latitude","Neighborhood Longitude","Venue Latitude","Venue Longitude","Cluster Labels"], axis=1, inplace=True)
toronto_counts1_1

,is_restaurant
Neighborhood,
Berczy Park,11
"Brockton, Parkdale Village, Exhibition Place",2
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",0
Central Bay Street,16
Christie,2
Church and Wellesley,26
"Commerce Court, Victoria Hotel",29
Davisville,10
Davisville North,0


Merge the 2 count table and see final result

In [187]:
toronto_label1 = pd.merge(toronto_counts_1, toronto_counts1_1, left_on = 'Neighborhood', right_on = 'Neighborhood')
toronto_label1

,Neighborhood,total_venues_count_per_neighboorhood,is_restaurant
0,Berczy Park,58,11
1,"Brockton, Parkdale Village, Exhibition Place",22,2
2,"CN Tower, King and Spadina, Railway Lands, Har...",17,0
3,Central Bay Street,61,16
4,Christie,16,2
5,Church and Wellesley,79,26
6,"Commerce Court, Victoria Hotel",100,29
7,Davisville,33,10
8,Davisville North,8,0
9,"Dufferin, Dovercourt Village",15,1


Calculate the restaurants of venues percentage and see the order

In [188]:
toronto_label1["percentage"] = toronto_label1["is_restaurant"]/toronto_label1["total_venues_count_per_neighboorhood"]

**Show the final result of the percentage calculating and conclude the result**

In [192]:
toronto_label1.sort_values(by=["percentage"], ascending=True, inplace= True)

**Below 5 neighborhoods are the best choice to open a new restaurant.**

In [193]:
toronto_label1.head()

,Neighborhood,total_venues_count_per_neighboorhood,is_restaurant,percentage
2,"CN Tower, King and Spadina, Railway Lands, Har...",17,0,0.000000
29,The Beaches,4,0,0.000000
8,Davisville North,8,0,0.000000
9,"Dufferin, Dovercourt Village",15,1,0.066667
1,"Brockton, Parkdale Village, Exhibition Place",22,2,0.090909


In [197]:
toronto_label1.loc[2, "Neighborhood"]

'CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport'

## Part5: Conclusion

### 1. In cluster 0, both neighborhoods Rosedale and The Danforth East are good choices to open a new restaurant, as there is no restaurant in top 10 venues yet.

### 2. In cluster 1, the top 5 neighborhoods for the choices to open a new restaurant are:
#### 1) "CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport", 
#### 2) "The Beaches", 
#### 3) "Davisville North"
#### 4) "Dufferin, Dovercourt Village"
#### 5) "Brockton, Parkdale Village, Exhibition Place"